In [124]:
import pandas as pd
import os

# fuzzy matching
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Table of Contents
- [Create Team Lists to Compare](#team-list-creation)
    - [Fuzzywuzzy to find matches](#fuzzywuzzy-to-find-matches)
- [Review Help](#review-help)
- [Final mapping creation](#final-mapping-creation)
- [Application of mapping](#application-of-mapping)

## Team List Creation

In [125]:
# read data from scotts free
odds_df = pd.read_csv('data/Package Sample 20240511/ncaaf_game_scores_1g_2022_sample.csv')
odds_df.head()

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
0,2022-23,2022-08-27,austin_peay,27,27.5,-112,0.0,western_kentucky,38,-27.5,-112,0.0,64.5,-109,-109
1,2022-23,2022-08-27,northwestern,31,11.0,-110,350.0,nebraska,28,-11.0,-110,-439.0,52.0,-110,-110
2,2022-23,2022-08-27,wyoming,6,14.0,-110,430.0,illinois,38,-14.0,-110,-550.0,43.0,-110,-110
3,2022-23,2022-08-27,idaho_state,21,23.0,-112,0.0,unlv,52,-23.0,-112,0.0,52.5,-109,-109
4,2022-23,2022-08-27,sf_austin,17,-6.5,-125,0.0,jacksonville_state,42,6.5,-105,0.0,51.5,-112,-112


In [126]:
# read data from ESPN
game_df = pd.read_csv('data/data_crawling/table_games/2022/games.csv', index_col=0)
game_df.head()

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record
0,401426532,https://www.espn.com/college-football/scoreboa...,NaN,Austin Peay,(0-10-1Away),NaN,Western Kentucky,(1-01-0Home)
1,401405059,https://www.espn.com/college-football/scoreboa...,NaN,Nebraska,(0-10-1Big Ten),NaN,Northwestern,(1-01-0Big Ten)
2,401413256,https://www.espn.com/college-football/scoreboa...,NaN,Idaho State,(0-10-1Away),NaN,UNLV,(1-01-0Home)
3,401404146,https://www.espn.com/college-football/scoreboa...,NaN,UConn,(0-10-1Away),NaN,Utah State,(1-01-0Home)
4,401405058,https://www.espn.com/college-football/scoreboa...,NaN,Wyoming,(0-10-1Away),NaN,Illinois,(1-01-0Home)


In [10]:
# make list of unique team names from ESPN using away_team_name and home_team_name from game_df and put into a list
espn_team_list = game_df['away_team_name'].unique().tolist() + game_df['home_team_name'].unique().tolist()

# now take the unique items in this list espn_team_list
espn_team_list = list(set(espn_team_list))

# make list of unique team names from odds_df using away_team and home_team and put into a list. Also remove any '_' with a space. This character is just going to make for a lower similiarty score
odds_team_list = odds_df['away_team'].unique().tolist() + odds_df['home_team'].unique().tolist()

replace_dict = {'_': ' ', '-': ' ', 'st': 'state'}

odds_team_list = [team.replace('_', ' ') for team in odds_team_list]
odds_team_list = [team.replace('-', ' ') for team in odds_team_list]
odds_team_list = list(set(odds_team_list))

### fuzzywuzzy to find matches

In [23]:
# # Create an empty list to store the mapping results
# mapping = []

# # Find the best match for each item in list1 within list2
# for item in espn_team_list:
#     best_matches = process.extract(item, odds_team_list, limit=3)  # You can adjust the limit based on how many matches you want
#     for match in best_matches:
#         mapping.append({
#             'Item from ESPN': item,
#             'Best Match from Odds Data': match[0],
#             'Similarity Score': match[1]
#         })

# # Convert the list of dictionaries to a DataFrame
# mapping_df = pd.DataFrame(mapping)
# mapping_df.head()

In [22]:
# # find in nebraska odds_team_list - can use this to check for any duplicates you make find.
# nebraska = [team for team in odds_team_list if 'nebraska' in team.lower()]
# nebraska

In [21]:
# # if score is 100, then it is a perfect match. Use this logic to make a column called 'Match' in mapping_df
# mapping_df['Match'] = mapping_df['Similarity Score'].apply(lambda x: 'Match' if x == 100 else 'No Match')

# mapping_df.Match.value_counts()

In [20]:
# # good matches
# perfect_match = mapping_df[mapping_df['Match'] == 'Match']
# perfect_match_teams = perfect_match['Item from ESPN'].unique().tolist()

# # filter these perfect match teams out of mapping_df
# mapping_df_review = mapping_df[~mapping_df['Item from ESPN'].isin(perfect_match_teams)]
# mapping_df_review.head()

### maybe this would make more sense to approach in reverse as we only care about the games where we have a team name in the first place

In [16]:
# Create an empty list to store the mapping results
mapping_reverse = []

# Find the best match for each item in list1 within list2
for item in odds_team_list:
    best_matches = process.extract(item, espn_team_list, limit=3)  # You can adjust the limit based on how many matches you want
    for match in best_matches:
        mapping_reverse.append({
            'Item from Odds Data': item,
            'Best Match from ESPN': match[0],
            'Similarity Score': match[1]
        })

# Convert the list of dictionaries to a DataFrame
mapping_df_reverse = pd.DataFrame(mapping_reverse)
mapping_df_reverse.head()

,Item from Odds Data,Best Match from ESPN,Similarity Score
0,penn state,Penn State,100
1,penn state,North Dakota State,86
2,penn state,Mississippi Valley State,86
3,appalachian st,Appalachian State,90
4,appalachian st,St. Thomas University (Fl),86


In [17]:
# if score is 100, then it is a perfect match. Use this logic to make a column called 'Match' in mapping_df
mapping_df_reverse['Match'] = mapping_df_reverse['Similarity Score'].apply(lambda x: 'Match' if x == 100 else 'No Match')

mapping_df_reverse.Match.value_counts()

Match
No Match    587
Match       214
Name: count, dtype: int64

In [18]:
# good matches
perfect_match_reverse = mapping_df_reverse[mapping_df_reverse['Match'] == 'Match']
perfect_match_reverse_teams = perfect_match_reverse['Item from Odds Data'].unique().tolist()

# filter these perfect match teams out of mapping_df
mapping_df_reverse_review = mapping_df_reverse[~mapping_df_reverse['Item from Odds Data'].isin(perfect_match_reverse_teams)]
mapping_df_reverse_review.head()

,Item from Odds Data,Best Match from ESPN,Similarity Score,Match
3,appalachian st,Appalachian State,90,No Match
4,appalachian st,St. Thomas University (Fl),86,No Match
5,appalachian st,St. Thomas - Minnesota,86,No Match
15,seton hall,Stonehill,74,No Match
16,seton hall,Elon,68,No Match


#### kick this file out to review manually

In [19]:
mapping_df_reverse_review.to_csv('data/data_crawling/table_team_mapping/mapping_df_reverse_review.csv', index=False)

### Review Help

In [25]:
# find any items in the espn_team_list that container 'missouri'
missouri = [team for team in espn_team_list if 'missouri' in team.lower()]
missouri

[]

In [118]:
# using above pattern we can use this to search through the options available
match_term = 'austin'

## search espn list
match_pattern = [team for team in espn_team_list if match_term in team.lower()]

# ## search odds list
# match_pattern = [team for team in odds_team_list if match_term in team.lower()]
match_pattern

['Stephen F. Austin', 'Austin Peay']

In [96]:
# find some examples from the game dataset to confirm the match
game_df[game_df['home_team_name'] == 'Miami']

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record
45,401411097,https://www.espn.com/college-football/scoreboa...,NaN,Bethune-Cookman,(0-10-1Away),16.0,Miami,(1-01-0Home)
439,401411105,https://www.espn.com/college-football/scoreboa...,NaN,Southern Miss,(0-20-1Away),15.0,Miami,(2-02-0Home)
1177,401426568,https://www.espn.com/college-football/scoreboa...,NaN,Middle Tennessee,(3-12-1Away),25.0,Miami,(2-22-1Home)
1728,401411133,https://www.espn.com/college-football/scoreboa...,NaN,North Carolina,(5-12-0ACC),NaN,Miami,(2-30-1ACC)
2147,401411144,https://www.espn.com/college-football/scoreboa...,NaN,Duke,(5-32-2ACC),NaN,Miami,(3-41-2ACC)
2614,401411153,https://www.espn.com/college-football/scoreboa...,NaN,Florida State,(6-34-3ACC),NaN,Miami,(4-52-3ACC)
3361,401411175,https://www.espn.com/college-football/scoreboa...,NaN,Pittsburgh,(8-45-3ACC),NaN,Miami,(5-73-5ACC)


In [108]:
odds_df[odds_df['home_team'] == 'seton_hall']
# odds_df[odds_df['away_team'] == 'portland_state']

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
831,2022-23,2022-11-05,wagner,10,14.0,-105,500.0,seton_hall,50,-13.5,-110,-550.0,48.0,-110,-110
1041,2022-23,2022-11-19,central_connecticut_state,39,6.5,102,245.0,seton_hall,14,-6.5,-112,-245.0,44.0,-109,-110


### Final mapping creation

In [110]:
updated_mapping_df = pd.read_csv('data/data_crawling/table_team_mapping/mapping_df_reverse_review_complete.csv')
updated_mapping_df.head()

,Item from Odds Data,Best Match from ESPN,Similarity Score,Match,Manual Map
0,appalachian st,Appalachian State,90,No Match,Match
1,appalachian st,St. Thomas University (Fl),86,No Match,No Match
2,appalachian st,St. Thomas - Minnesota,86,No Match,No Match
3,seton hall,Stonehill,74,No Match,Match
4,seton hall,Elon,68,No Match,No Match


In [111]:
updated_mapping_df_manual = updated_mapping_df[updated_mapping_df['Manual Map'] == 'Match']
updated_mapping_df_manual.head()

,Item from Odds Data,Best Match from ESPN,Similarity Score,Match,Manual Map
0,appalachian st,Appalachian State,90,No Match,Match
3,seton hall,Stonehill,74,No Match,Match
6,n. dakota state,North Dakota State,87,No Match,Match
12,se missouri state,Southeast Missouri State,0,No Match,Match
14,louisiana lafayette,Louisiana,90,No Match,Match


In [113]:
perfect_match_reverse['Manual Map'] = ''
perfect_match_reverse.head()

C:\Users\rober\AppData\Local\Temp\ipykernel_4344\1632818970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perfect_match_reverse['Manual Map'] = ''


,Item from Odds Data,Best Match from ESPN,Similarity Score,Match,Manual Map
0,penn state,Penn State,100,Match,
6,old dominion,Old Dominion,100,Match,
9,stony brook,Stony Brook,100,Match,
12,akron,Akron,100,Match,
18,western kentucky,Western Kentucky,100,Match,


In [114]:
# final mapping df
final_mapping_df = pd.concat([perfect_match_reverse, updated_mapping_df_manual], axis=0)
final_mapping_df.head()

,Item from Odds Data,Best Match from ESPN,Similarity Score,Match,Manual Map
0,penn state,Penn State,100,Match,
6,old dominion,Old Dominion,100,Match,
9,stony brook,Stony Brook,100,Match,
12,akron,Akron,100,Match,
18,western kentucky,Western Kentucky,100,Match,


In [115]:
final_mapping_df.to_csv('data/data_crawling/table_team_mapping/final_mapping_df.csv', index=False)

### Application of mapping

In [133]:
# replace '_' in away_team and home_team columns in odds_df with ' '
odds_df_mapped = odds_df.copy(deep=True)
odds_df_mapped['away_team'] = odds_df_mapped['away_team'].str.replace('_', ' ')
odds_df_mapped['home_team'] = odds_df_mapped['home_team'].str.replace('_', ' ')
odds_df_mapped.head()

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
0,2022-23,2022-08-27,austin peay,27,27.5,-112,0.0,western kentucky,38,-27.5,-112,0.0,64.5,-109,-109
1,2022-23,2022-08-27,northwestern,31,11.0,-110,350.0,nebraska,28,-11.0,-110,-439.0,52.0,-110,-110
2,2022-23,2022-08-27,wyoming,6,14.0,-110,430.0,illinois,38,-14.0,-110,-550.0,43.0,-110,-110
3,2022-23,2022-08-27,idaho state,21,23.0,-112,0.0,unlv,52,-23.0,-112,0.0,52.5,-109,-109
4,2022-23,2022-08-27,sf austin,17,-6.5,-125,0.0,jacksonville state,42,6.5,-105,0.0,51.5,-112,-112


In [134]:
## lets use the mapping to provide a normalized team name for away and home team names
# use final_mapping_df to create a dictionary of team names you can use Item from Odds Data and Best Match from ESPN columns
team_mapping_dict = final_mapping_df.set_index('Item from Odds Data')['Best Match from ESPN'].to_dict()

# now use this dictionary to replace the team names in the odds_df
odds_df_mapped['away_team'] = odds_df_mapped['away_team'].map(team_mapping_dict)
odds_df_mapped['home_team'] = odds_df_mapped['home_team'].map(team_mapping_dict)

odds_df_mapped.head()

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
0,2022-23,2022-08-27,Austin Peay,27,27.5,-112,0.0,Western Kentucky,38,-27.5,-112,0.0,64.5,-109,-109
1,2022-23,2022-08-27,Northwestern,31,11.0,-110,350.0,Nebraska,28,-11.0,-110,-439.0,52.0,-110,-110
2,2022-23,2022-08-27,Wyoming,6,14.0,-110,430.0,Illinois,38,-14.0,-110,-550.0,43.0,-110,-110
3,2022-23,2022-08-27,Idaho State,21,23.0,-112,0.0,UNLV,52,-23.0,-112,0.0,52.5,-109,-109
4,2022-23,2022-08-27,Stephen F. Austin,17,-6.5,-125,0.0,Jacksonville State,42,6.5,-105,0.0,51.5,-112,-112


In [128]:
# now lets make a lookup to the game_df

# create a column that counts the number of times each team plays eachother
game_df['matchup_count'] = game_df.groupby(['away_team_name', 'home_team_name']).cumcount()
game_df.head()

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record,matchup_count
0,401426532,https://www.espn.com/college-football/scoreboa...,NaN,Austin Peay,(0-10-1Away),NaN,Western Kentucky,(1-01-0Home),0
1,401405059,https://www.espn.com/college-football/scoreboa...,NaN,Nebraska,(0-10-1Big Ten),NaN,Northwestern,(1-01-0Big Ten),0
2,401413256,https://www.espn.com/college-football/scoreboa...,NaN,Idaho State,(0-10-1Away),NaN,UNLV,(1-01-0Home),0
3,401404146,https://www.espn.com/college-football/scoreboa...,NaN,UConn,(0-10-1Away),NaN,Utah State,(1-01-0Home),0
4,401405058,https://www.espn.com/college-football/scoreboa...,NaN,Wyoming,(0-10-1Away),NaN,Illinois,(1-01-0Home),0


#### verifying this idea actual works. Instead of getting the date behind each game I think we can just use this matchup count concept

In [130]:
game_df[game_df['matchup_count'] == 1]

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record,matchup_count
3461,401506450,https://www.espn.com/college-football/scoreboa...,NaN,Akron,(2-101-7MAC),NaN,Buffalo,(6-65-3MAC),1
3462,401437032,https://www.espn.com/college-football/scoreboa...,NaN,North Texas,(7-66-2CUSA),NaN,UTSA,(11-28-0CUSA),1
3465,401437030,https://www.espn.com/college-football/scoreboa...,10.0,Kansas State,(10-37-2Big 12),3.0,TCU,(12-19-0Big 12),1
3467,401437008,https://www.espn.com/college-football/scoreboa...,22.0,UCF,(9-46-2American),18.0,Tulane,(11-27-1American),1
3470,401437034,https://www.espn.com/college-football/scoreboa...,NaN,Fresno State,(9-47-1Mountain West),NaN,Boise State,(9-48-0Mountain West),1
3488,401506611,https://www.espn.com/college-football/scoreboa...,NaN,Weber State,(10-36-2Big Sky),NaN,Montana State,(11-18-0Big Sky),1
3491,401437132,https://www.espn.com/college-football/scoreboa...,NaN,Southern,(7-55-3SWAC),NaN,Jackson State,(12-08-0SWAC),1


In [132]:
game_df[game_df['away_team_name'] == 'Kansas State']

,id,url,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record,matchup_count
1162,401404081,https://www.espn.com/college-football/scoreboa...,NaN,Kansas State,(3-11-0Big 12),6.0,Oklahoma,(3-10-1Big 12),0
1706,401404088,https://www.espn.com/college-football/scoreboa...,20.0,Kansas State,(5-13-0Big 12),NaN,Iowa State,(3-30-3Big 12),0
2135,401404097,https://www.espn.com/college-football/scoreboa...,17.0,Kansas State,(5-23-1Big 12),8.0,TCU,(7-04-0Big 12),0
2819,401404109,https://www.espn.com/college-football/scoreboa...,19.0,Kansas State,(7-35-2Big 12),NaN,Baylor,(6-44-3Big 12),0
3064,401404117,https://www.espn.com/college-football/scoreboa...,15.0,Kansas State,(8-36-2Big 12),NaN,West Virginia,(4-72-6Big 12),0
3465,401437030,https://www.espn.com/college-football/scoreboa...,10.0,Kansas State,(10-37-2Big 12),3.0,TCU,(12-19-0Big 12),1


In [135]:
# apply this same matchup count to the odds_df_mapped
odds_df_mapped['matchup_count'] = odds_df_mapped.groupby(['away_team', 'home_team']).cumcount()
odds_df_mapped.head()

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,matchup_count
0,2022-23,2022-08-27,Austin Peay,27,27.5,-112,0.0,Western Kentucky,38,-27.5,-112,0.0,64.5,-109,-109,0.0
1,2022-23,2022-08-27,Northwestern,31,11.0,-110,350.0,Nebraska,28,-11.0,-110,-439.0,52.0,-110,-110,0.0
2,2022-23,2022-08-27,Wyoming,6,14.0,-110,430.0,Illinois,38,-14.0,-110,-550.0,43.0,-110,-110,0.0
3,2022-23,2022-08-27,Idaho State,21,23.0,-112,0.0,UNLV,52,-23.0,-112,0.0,52.5,-109,-109,0.0
4,2022-23,2022-08-27,Stephen F. Austin,17,-6.5,-125,0.0,Jacksonville State,42,6.5,-105,0.0,51.5,-112,-112,0.0


In [136]:
## just further confirm this might work
odds_df_mapped[odds_df_mapped['away_team'] == 'Kansas State']

,season,date,away_team,away_score,away_point_spread,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,matchup_count
276,2022-23,2022-09-24,Kansas State,41,13.5,-110,402.0,Oklahoma,34,-13.5,-110,-500.0,53.0,-109,-110,0.0
394,2022-23,2022-10-08,Kansas State,10,-1.0,-109,-117.0,Iowa State,9,1.0,-111,-103.0,45.0,-110,-110,0.0
607,2022-23,2022-10-22,Kansas State,28,3.5,-110,150.0,TCU,38,-3.0,-110,-143.0,54.5,-108,-105,0.0
942,2022-23,2022-11-12,Kansas State,31,2.5,-105,120.0,Baylor,3,-2.5,-110,-135.0,52.5,-105,-105,0.0
1077,2022-23,2022-11-19,Kansas State,48,-8,-110,-295.0,West Virginia,31,8.0,-105,265.0,54.5,-110,-104,0.0
1169,2022-23,2022-12-03,Kansas State,31,-1.0,-109,-114.0,TCU,28,1.5,-105,100.0,60.5,-110,-110,1.0


In [138]:
# join odds_df_mapped and game_df on away_team and home_team and matchup_count
merged_df = pd.merge(game_df, odds_df_mapped, how='inner', left_on=['away_team_name', 'home_team_name', 'matchup_count'], right_on=['away_team', 'home_team', 'matchup_count'])

# drop url, and matchup_count columns
merged_df.drop(columns=['url', 'matchup_count'], inplace=True)
merged_df.head()

,id,away_rank,away_team_name,away_record,home_rank,home_team_name,home_record,season,date,away_team,...,away_point_spread_line,away_money_line,home_team,home_score,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line
0,401426532,NaN,Austin Peay,(0-10-1Away),NaN,Western Kentucky,(1-01-0Home),2022-23,2022-08-27,Austin Peay,...,-112,0.0,Western Kentucky,38,-27.5,-112,0.0,64.5,-109,-109
1,401413256,NaN,Idaho State,(0-10-1Away),NaN,UNLV,(1-01-0Home),2022-23,2022-08-27,Idaho State,...,-112,0.0,UNLV,52,-23.0,-112,0.0,52.5,-109,-109
2,401405058,NaN,Wyoming,(0-10-1Away),NaN,Illinois,(1-01-0Home),2022-23,2022-08-27,Wyoming,...,-110,430.0,Illinois,38,-14.0,-110,-550.0,43.0,-110,-110
3,401411090,NaN,Duquesne,(0-10-1Away),NaN,Florida State,(1-01-0Home),2022-23,2022-08-27,Duquesne,...,-105,0.0,Florida State,47,-43.5,-125,0.0,54.5,-112,-112
4,401426530,NaN,Charlotte,(0-10-1CUSA),NaN,Florida Atlantic,(1-01-0CUSA),2022-23,2022-08-27,Charlotte,...,-110,235.0,Florida Atlantic,43,-7.0,-110,-275.0,59.0,-110,-110


In [141]:
print("Pre Join: aka Games from Odds data:", len(odds_df_mapped))
print("Post Join: aka Matched Games:", len(merged_df))

Pre Join: aka Games from Odds data: 1224
Post Join: aka Matched Games: 1145


In [143]:
# check if there are any duplicate game ids
merged_df['id'].duplicated().sum()

0

In [144]:
merged_df.to_csv('data/pre_processed/odds_w_game_id.csv', index=False)